In [8]:
# ============================================
# УНИВЕРСАЛЬНЫЙ БЛОК:
# 1) читает PDF
# 2) через OpenAI вытаскивает содержание (RU/EN)
# 3) фильтрует только главы верхнего уровня
# 4) режет PDF на главы и сохраняет отдельными файлами
# ============================================

import os
import re
import json
import subprocess
import sys
from pathlib import Path



import fitz  # PyMuPDF
from openai import OpenAI

# ---------- ключ и базовые настройки ----------
OPENAI_API_KEY = ""   # <- ты сюда просто ставишь строку
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

PDF_PATH = "Gas and Oil Reliability Engineering - Modeling and Analysis -- Eduardo Calixto -- ( WeLib.org ).pdf"            # <- путь к твоей книге PDF
pdf_path = Path(PDF_PATH).resolve()
if not pdf_path.exists():
    raise FileNotFoundError(f"PDF не найден: {pdf_path}")

print("Книга:", pdf_path)

# сколько первых страниц отдавать модели, если придётся идти в LLM
N_PAGES_FOR_TOC = 20

# ---------- чтение текста страниц и bookmarks ----------
def extract_pages_and_bookmarks(pdf_file: Path):
    doc = fitz.open(pdf_file)
    pages = [doc[i].get_text("text") for i in range(len(doc))]
    # format: [ [level, title, page, ...], ... ]
    toc_bookmarks = doc.get_toc(simple=True)  # simple=True -> [level, title, page]
    doc.close()
    return pages, toc_bookmarks

pages_text, pdf_toc = extract_pages_and_bookmarks(pdf_path)
print("Всего страниц в PDF:", len(pages_text))
print("Количество записей в bookmarks (PDF TOC):", len(pdf_toc))

# ---------- универсальный фильтр названий глав ----------
def is_top_level_title(title: str) -> bool:
    t = title.strip()
    low = t.lower()

    # ключевые слова для глав/частей
    top_keywords_prefix = [
        "глава", "часть", "section", "chapter", "part"
    ]
    special_starts = [
        "введение", "introduction",
        "заключение", "conclusion",
        "эпилог", "epilogue",
        "литература", "список литературы",
        "references", "bibliography"
    ]

    for kw in top_keywords_prefix:
        if low.startswith(kw + " "):
            return True

    for kw in special_starts:
        if low.startswith(kw):
            return True

    return False

# ---------- 1) Пытаемся использовать bookmarks ----------
chapters_raw = []
toc_source = None

if pdf_toc:
    toc_source = "bookmarks"
    print("\nИспользуем TOC из bookmarks PDF.")
    # pdf_toc: [ [level, title, page], ... ]
    for level, title, page in pdf_toc:
        if not isinstance(page, int) or page <= 0:
            continue
        level_label = "chapter" if level == 1 else "section"
        chapters_raw.append(
            {
                "title": str(title).strip(),
                "page_number": page,   # это уже реальные PDF-страницы (1-базовые)
                "level": level_label
            }
        )

# ---------- 2) Если bookmarks пусты – используем OpenAI для извлечения TOC ----------
if not chapters_raw:
    toc_source = "llm"
    print("\nBookmarks пусты или не найдены. Переходим к извлечению содержания через OpenAI.")

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError(
            "OPENAI_API_KEY не задан в окружении, а bookmarks отсутствуют.\n"
            "Либо задай ключ, либо обеспечь оглавление в bookmarks."
        )

    client = OpenAI(api_key=api_key)

    def build_toc_prompt(pages):
        parts = []
        for i, txt in enumerate(pages, 1):
            parts.append(f"=== PAGE {i} ===\n{txt[:3500]}")
        block = "\n\n".join(parts)

        prompt = (
            "Ты анализируешь первые страницы книги на русском или английском языке.\n"
            "Найди содержание (Содержание / Contents / Table of contents) и сформируй структуру пунктов.\n\n"
            "Верни СТРОГО JSON без пояснений, в формате:\n"
            "{\n"
            '  \"chapters\": [\n'
            '    {\"title\": \"…\", \"page_number\": 10, \"level\": \"chapter\"},\n'
            '    {\"title\": \"…\", \"page_number\": 12, \"level\": \"section\"}\n'
            "  ]\n"
            "}\n\n"
            "Где:\n"
            "  - title — текст пункта из содержания, без точек-лидеров и номера страницы\n"
            "  - page_number — номер страницы, КАК УКАЗАНО В СОДЕРЖАНИИ (целое число)\n"
            "  - level — один из: \"chapter\" (глава верхнего уровня), "
            "\"section\" (подраздел), \"other\" (прочее)\n\n"
            "Важно:\n"
            "  1) Не придумывай лишних пунктов – только то, что реально есть в содержании.\n"
            "  2) Если нет явного слова 'Chapter'/'Глава', но заголовок типа "
            "'Введение', 'Заключение', 'Литература', 'Introduction', "
            "'Conclusion', 'References' – пометь это level = \"chapter\".\n"
            "  3) Выводи ТОЛЬКО JSON, без текста вокруг.\n\n"
            "Текст для анализа:\n\n"
            f"{block}"
        )
        return prompt

    n = min(N_PAGES_FOR_TOC, len(pages_text))
    prompt = build_toc_prompt(pages_text[:n])

    resp = client.responses.create(
        model="gpt-4.1-mini",
        input=prompt
    )

    raw_answer = resp.output_text
    print("\nСырой ответ модели:\n", raw_answer)

    start = raw_answer.find("{")
    end = raw_answer.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("Модель не вернула JSON-подобный фрагмент.")

    raw_json = raw_answer[start:end+1]

    try:
        toc = json.loads(raw_json)
    except json.JSONDecodeError as e:
        print("Ошибка парсинга JSON:", e)
        print("Сырый JSON-фрагмент:\n", raw_json)
        raise

    if not isinstance(toc.get("chapters"), list):
        raise RuntimeError("В JSON нет корректного списка 'chapters'.")

    for item in toc["chapters"]:
        title = str(item.get("title", "")).strip()
        page_number = item.get("page_number", None)
        level = str(item.get("level", "")).strip().lower()
        if not title or not isinstance(page_number, int):
            continue
        chapters_raw.append(
            {
                "title": title,
                "page_number": page_number,   # пока это логическая страница из содержания
                "level": level or "other"
            }
        )

print(f"\nИсточник TOC: {toc_source}")
print("Всего пунктов в raw TOC:", len(chapters_raw))

if not chapters_raw:
    raise RuntimeError("Не удалось извлечь никакого TOC ни из bookmarks, ни из LLM.")

# ---------- фильтрация только глав верхнего уровня ----------

chapters_filtered = []
for item in chapters_raw:
    title = item["title"]
    page_number = int(item["page_number"])
    level = str(item.get("level", "")).lower()

    # если модель/outline уже пометили как chapter — верим
    if level == "chapter":
        chapters_filtered.append({"title": title, "page_number": page_number})
        continue

    # если явно похоже на главу / крупный раздел — берём
    if is_top_level_title(title):
        chapters_filtered.append({"title": title, "page_number": page_number})

# сортируем по номеру страницы
chapters_filtered.sort(key=lambda x: x["page_number"])

if not chapters_filtered:
    raise RuntimeError("После фильтрации не осталось ни одной главы.")

print("\nГлавы верхнего уровня после фильтрации:")
for ch in chapters_filtered:
    print(f"- {ch['title']} (стр. {ch['page_number']})")

# ---------- определяем PAGE_OFFSET ----------
# Для bookmarks страницы уже реальные PDF-страницы -> смещение 0.
# Для LLM это логические страницы из содержания -> PAGE_OFFSET можно калибровать вручную.

if toc_source == "bookmarks":
    PAGE_OFFSET = 0
else:
    # базово считаем, что номера совпадают; при необходимости руками меняешь KNOWN_PDF_PAGE
    KNOWN_TOC_PAGE = chapters_filtered[0]["page_number"]
    KNOWN_PDF_PAGE = chapters_filtered[0]["page_number"]  # если нужно смещение — меняешь это число

    PAGE_OFFSET = KNOWN_PDF_PAGE - KNOWN_TOC_PAGE

print(f"\nИтоговый PAGE_OFFSET = {PAGE_OFFSET}  (источник TOC: {toc_source})")

# ---------- подготовка папки для глав ----------
def safe_filename(text: str) -> str:
    text = text.strip()
    text = re.sub(r"[^\w\s\-_]", "_", text)
    text = re.sub(r"\s+", " ", text)
    return text or "chapter"

output_dir = pdf_path.parent / pdf_path.stem
os.makedirs(output_dir, exist_ok=True)
print("Папка для глав:", output_dir)

# ---------- разбиение PDF на главы ----------
def split_pdf_into_chapters(pdf_file: Path, chapters_list, out_dir: Path, page_offset: int = 0):
    doc = fitz.open(pdf_file)
    total_pages = len(doc)
    print(f"\nВсего страниц в PDF: {total_pages}\n")

    for idx, ch in enumerate(chapters_list):
        logical_page = ch["page_number"]
        start_page = logical_page - 1 + page_offset  # 0-базовый индекс

        if idx + 1 < len(chapters_list):
            next_logical = chapters_list[idx + 1]["page_number"]
            end_page = next_logical - 1 + page_offset
        else:
            end_page = total_pages

        start_page = max(0, min(start_page, total_pages - 1))
        end_page = max(start_page + 1, min(end_page, total_pages))

        new_doc = fitz.open()
        for p in range(start_page, end_page):
            new_doc.insert_pdf(doc, from_page=p, to_page=p)

        filename = f"{idx + 1:02d} - {safe_filename(ch['title'])}.pdf"
        out_path = out_dir / filename
        new_doc.save(out_path)
        new_doc.close()

        print(
            f"Глава {idx + 1}: '{ch['title']}'\n"
            f"  логическая страница: {logical_page}\n"
            f"  реальные PDF-страницы: {start_page + 1} – {end_page}\n"
            f"  файл: {out_path.name}\n"
        )

    doc.close()
    print("Разбиение завершено.\n")

split_pdf_into_chapters(pdf_path, chapters_filtered, output_dir, PAGE_OFFSET)

print("Содержимое папки с главами:")
for f in sorted(output_dir.iterdir()):
    print(" •", f.name)

Книга: /Users/ivan/code/chapter_splitter/Gas and Oil Reliability Engineering - Modeling and Analysis -- Eduardo Calixto -- ( WeLib.org ).pdf
Всего страниц в PDF: 795
Количество записей в bookmarks (PDF TOC): 286

Используем TOC из bookmarks PDF.

Источник TOC: bookmarks
Всего пунктов в raw TOC: 286

Главы верхнего уровня после фильтрации:
- Cover (стр. 1)
- Gas and Oil Reliability Engineering: Modeling and Analysis (стр. 2)
- © (стр. 3)
- Dedication (стр. 4)
- Preface (стр. 5)
- Acknowledgment (стр. 9)
- 1 . Lifetime Data Analysis (стр. 11)
- References (стр. 102)
- 2 . Accelerated Life Test, Reliability Growth Analysis, and Probabilistic Degradation Analysis (стр. 103)
- References (стр. 168)
- 3 . Reliability and Maintenance (стр. 169)
- References (стр. 229)
- 4 . Reliability, Availability, and Maintainability (стр. 278)
- References (стр. 478)
- Bibliography (стр. 479)
- 5 . Human Reliability Analysis (стр. 480)
- References (стр. 560)
- 6 . Reliability and Safety Processes (стр. 5